# Solución de la ecuación $\boldsymbol{q} = \boldsymbol{K}\boldsymbol{a} - \boldsymbol{f}$ mediante ensamblaje matricial.

| Nombre | Correo | Fecha |
| --- | --- | --- |
| Andrés Cardona Serna | acardonase@unal.edu.co | 25 de agosto de 2023 |

## 1. Problema

Considere la siguiente estructura unidimensional de tres barras:

<img src="Tres_Barras_1D.jpeg"/>

Cada una de las barras tiene dos nodos (con un grado de libertad cada uno) para un total de 4 g.d.l., la misma área transversal $A$ y el mismo módulo de elasticidad $E$.

A continuación se hallarán, mediante ensamblaje matricial:
- La matriz de rigidez global $\boldsymbol{K}$ de la estructura.
- El vector de fuerzas nodales equivalentes global $\boldsymbol{f}$.
  
Luego se planteará el sistema de ecuaciones $\boldsymbol{q} = \boldsymbol{K}\boldsymbol{a} - \boldsymbol{f}$, para resolverlo, y encontrar:
- El vector de desplazamientos nodales desconocidos $\boldsymbol{a}_d$
- El vector de fuerzas nodales de equilibrio desconocidas $\boldsymbol{q}_d$.
  
Para finalmente volver a ensamblar y obtener:
- El vector de desplazamientos nodales global $\boldsymbol{a}$.
- El vector de fuerzas nodales de equilibrio $\boldsymbol{q}$.
  
Y adicionalmente, se encontrará la fuerza axial $f_{ax}$ de cada barra.

Nota: Ejecute cada celda en orden desde la primera hasta la última, o de lo contrario, el códigos podría calcular resultados erróneos.

---

In [1]:
import numpy as np    # Librería de cálculo simbólico
import sympy as sp    # Librería de cálculo numérico

# Para imprimir bonito
sp.init_printing()
from IPython.display import Math
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{sp.latex(variable)}' + texto2)

In [2]:
# Ya que en Python el conteo comienza desde 0, se definen variables que hacen más legible el código
NL1 = 0               # en referencia al nodo local 1
NL2 = 1               # en referencia al nodo local 2

---

## 2. Ensamblaje de $\boldsymbol{f}$ y $\boldsymbol{K}$.

### 2.1. Funciones para el ensamblaje.

Se definen dos funciones que se utilizarán más adelante: una para ensamblar la matriz de rigidez global $\boldsymbol{K}$ y otra para ensamblar el vector de fuerzas nodales equivalentes global $\boldsymbol{f}$. A continuación se explica de manera muy general, pero más adelante se verán ejemplos que aclaran el uso de estas funciones.

- La función `ensamblar_mat` ensambla la matriz de rigidez local de la barra $e$ $(\boldsymbol{K}^{(e)})$ en la matriz de rigidez global   $(\boldsymbol{K})$.  
  Se invoca mediante el comando `ensamblar_mat(K, idx, Ke)`, y sus parámetros de entrada son:
    - `K` : Matriz de rigidez global $(\boldsymbol{K})$.
    - `idx` : Vector fila con los g.d.l. donde se debe realizar el ensamblaje.
    - `Ke` : Matriz de rigidez local de la barra $e$, que se va a ensamblar $(\boldsymbol{K}^{(e)})$.

In [3]:
def ensamblar_mat(K, idx, Ke):                 

    # Se verifica que la matriz global K sea cuadrada
    nfil, ncol = K.shape                                            # Dada K de tamaño nxm, K.shape es la tupla (n,m), por lo que nfil=n y ncol=m.
    assert nfil == ncol,\
       "La matriz de rigidez global K debe ser cuadrada"       
                                                                    # Si nfil = ncol, K es cuadrada. Se arroja un error en caso contrario.

    

    # Se verifica que la matriz local Ke sea cuadrada y con el mismo número de filas que idx
    nfil, ncol = Ke.shape             
    assert nfil == ncol == len(idx),\
       "Ke debe ser cuadrada y debe tener el mismo número de filas que idx"
                                                   # Si el vector idx contiene m grados de libertad, Ke debe ser de tamaño mxm. Error en caso contrario.


    
    # Se procede con el ensamblaje               # Si la matriz Ke es de tamaño mxm:
    for i in range(nfil):                        # Se ejecuta una iteración para cada una de las m filas de Ke
        for j in range(ncol):                    # Se ejecuta una iteración para cada una de las m columnas de Ke. El ciclo anidado itera m*m veces.
            K[idx[i], idx[j]] += Ke[i,j]               # Se ensamblan en la matriz global K los elementos de la matriz local Ke.

- La función `ensamblar_vec` extrae las fuerzas nodales equivalentes de las barras y las ensambla en el vector de fuerzas nodales equivalentes global $\boldsymbol{f}$.
    Se invoca mediante el comando `ensamblar_vec(v, idx, ve)`, donde:
    - `v` : Vector global.
    - `idx` : Vector fila con los g.d.l. en que se debe realizar el ensamblaje.
    - `ve` : Vector local que se va a ensamblar.

In [4]:
# Para ensamblar vector f
def ensamblar_vec(v, idx, ve):

    # Se verifica que v sea un vector columna
    nfil, ncol = v.shape
    assert ncol == 1, "v debe ser un vector columna"           
                                                # El número de columnas de v debe ser igual a 1. Error en caso contrario.
    

    
    # Se verifica que el ve sea vector columna y que tenga el mismo número de elementos que idx
    nfil, ncol = ve.shape
    assert ncol == 1 and nfil == len(idx),\
            "ve debe ser vector columna y debe tener el mismo número de elementos que idx"
                                                # Si el vector idx contiene m grados de libertad, ve debe tener m elementos. Error en caso contrario.

    
    
    # se procede con el ensamblaje              # Si el vector ve es de tamaño mx1:
    for i in range(nfil):                       # Se ejecuta una iteración para cada una de las m filas de ve.
        f[idx[i]] += ve[i]                # Se ensamblan en el vector global v los elementos del vector local ve.

---

### 2.2. Definición de variables y datos.

- Se definen las variables simbólicas necesarias:  
  - Carga distribuida constante $b$.  
  - Módulo de elasticidad $E$.  
  - Área transversal $A$.  
  - Longitud $L$.   
  - Carga puntual aplicada $P$.  

In [5]:
b, E, A, L, P = sp.symbols('b E A L P')         # define las variables simbólicas

- Se define la matriz LaG (local a global). Contiene la numeración global de cada nodo, según su numeración local y la barra en que se encuentre:
<center><img src="Matriz_LaG.jpeg" style="height: 117px; width:294px;"/></center>

In [6]:
# Se le resta 1 a la matriz, pues el conteo en Python comienza desde 0. Esto le restará 1 a cada componente de la matriz.

              # NL1, NL2
LaG = np.array([[ 1, 3 ],            # Barra 1
                [ 2, 3 ],            # Barra 2
                [ 3, 4 ]]) - 1       # Barra 3

- Se definen las longitudes $L^{(e)}$ para cada barra:

In [7]:
#  e =  1,  2,  3
long = [L,  L,  L/2]          # Lista con la longitud de cada barra

In [8]:
imprimir(r" L^{(1)} = ", long[1-1])

<IPython.core.display.Math object>

In [9]:
imprimir(r" L^{(2)} = ", long[2-1])

<IPython.core.display.Math object>

In [10]:
imprimir(r" L^{(3)} = ", long[3-1])

<IPython.core.display.Math object>

---

### 2.3. Cálculo de las rigideces axiales

Se calculan las rigideces axiales $k^{(e)}$ para cada barra, por medio de la ecuación 
$$k^{(e)} = \frac{E^{(e)} A^{(e)}} {L^{(e)}}.$$ 
Como $E^{(e)}$ y $A^{(e)}$ son constantes en las tres barras, solo se varía el valor $L^{(e)}$ de cada barra. Entonces las rigideces de las tres barras quedarán almacenadas en la lista `k`:

In [11]:
k = [E*A/longitud for longitud in long]         # Se crea la lista k = [ E*A/long[1] , E*A/long[2] , E*A/long[3] ]
imprimir("", k)

<IPython.core.display.Math object>

In [12]:
imprimir(r" k^{(1)} = " , k[1-1])

<IPython.core.display.Math object>

In [13]:
imprimir(r" k^{(2)} = " , k[2-1])

<IPython.core.display.Math object>

In [14]:
imprimir(r" k^{(3)} = " , k[3-1])

<IPython.core.display.Math object>

---

### 2.4. Ensamblaje "a pedal".

A continuación se ensamblará un elemento en $\boldsymbol{K}$ y un elemento en $\boldsymbol{f}$ utilizando las mismas líneas de código del ensamblaje automático, con la diferencia de que se analizará cada línea y se explicará al menos una iteración de cada ciclo que se utilice.

#### 2.4.1. Se definen $\boldsymbol{f}$ y $\boldsymbol{K}$.

- Como la estructura tiene cuatro g.d.l, la matriz de rigidez global $\boldsymbol{K}$ será de tamaño 4x4, así que se define inicialmente una matriz de ceros:

In [15]:
K = sp.zeros(4)                               # Separa memoria para matriz de rigidez global K de tamaño 4x4
imprimir(r"\boldsymbol{K} = ", K)

<IPython.core.display.Math object>

- De igual manera, como la estructura tiene cuatro g.d.l, el vector de fuerzas nodales equivalentes global $\boldsymbol{f}$ será de tamaño 4x1, así que se define inicialmente un vector de ceros y se le agregan las cargas puntuales en los nodos:

In [16]:
f = sp.zeros(4,1)     # Se separa memoria para el vector f global de tamaño 4x1
f[3-1] = P/2          # Se agrega la carga puntual aplicada en el g.d.l. 3
f[4-1] = P            # Se agrega la carga puntual aplicada en el g.d.l. 4

In [17]:
imprimir(r" \boldsymbol{f} = ", f)

<IPython.core.display.Math object>

- Se define una matriz llamada `fe` que contiene las fuerzas nodales equivalentes para cada barra. La columna $e$ representa la barra número #$e$,   mientras que las filas $1$ y $2$ indican si la fuerza nodal se encuentra en el nodo local $1$ o en el nodo local $2$, respectivamente.  

  O dicho de otra forma, la columna $e$ es el vector local de fuerzas nodales equivalentes de la barra #$e$, denotado como $\boldsymbol{f}^{(e)}$:

In [18]:
          #  e  =   1  ,   2  ,  3
fe = sp.Matrix([[ b*L/2, b*L/2,  0 ],    # NL1
                [ b*L/2, b*L/2,  0 ]])   # NL2

---

#### 2.4.2. Ciclo `for`.

Se ensambla la matriz de rigidez global $\boldsymbol{K}$ y el vector global $\boldsymbol{f}$ con de las siguientes líneas de código:

```
for e in range(3):                              # Se ejecuta una iteración para cada barra e = 1, 2 y 3 (0, 1 y 2 en Python).
    idx = LaG[e,:]                              # Vector fila con índices de los nodos globales de la barra e.
    Ke  = k[e]*np.array([[1, -1],[-1, 1]])      # Matriz de rigidez local de la barra e.
    ensamblar_mat(K, idx, Ke)                   # Se ensambla Ke en la matriz de rigidez global K.
    ensamblar_vec(f, idx, fe[:,e])              # Se ensamblan las fuerzas nodales equivalentes de la barra e en el vector global f.
```
Veamos línea por línea la iteración del ciclo `for` de la barra $e=2$ ($e=1$ en Python), para entender cómo funciona el ensamblaje:

- Se define el vector `idx`, el cual extraerá de la matriz `LaG` los nodos globales de la barra $e=2$ ($e=1$ en  Python). Es decir, extraerá los nodos $2$ y $3$ ($1$ y $2$ en Python):

In [19]:
e = 2-1                                  # Se define e=2 (e=1 en Python)
idx = LaG[e, :]                          # Extrae la fila 2 (correspondiente a la barra 2) de la matriz LaG,o sea, la matriz [2  3]
imprimir(r"\mathrm{idx} = ", idx+1)      # Se le suma 1 para que se imprima como si el conteo fuera desde 1.

<IPython.core.display.Math object>

- Se define la matriz  de rigidez local de la barra $e=2$, por medio de la ecuación 
                   $$\boldsymbol{K}^{(e)} = k^{(e)} \begin{bmatrix}  1  &  -1  \\  -1  &  1  \end{bmatrix}.$$

  Recuerde que la rigidez axial $k^{(2)}$ se encuentra en la posición #$2$ (#$1$ en Python) de la lista ya definida como `k`:

In [20]:
Ke  = k[e]*sp.Matrix([[1, -1],[-1, 1]])                    # k[2-1] extrae la rigidez de la barra 2.
imprimir(r"\boldsymbol{K}^{(2)} = ", Ke)

<IPython.core.display.Math object>

##### 2.4.2.1. Función `ensamblar_mat` para $\boldsymbol{K}$.

En la siguiente línea se utiliza la función ya definida como `ensamblar_mat(K, idx, Ke)`, pero en lugar de ejecutarla, vamos a ejecutar las líneas que hay dentro de la función:  

```
# Se verifica que la matriz global K sea cuadrada
nfil, ncol = K.shape                                                  
assert nfil == ncol, "La matriz de rigidez K debe ser cuadrada"       

                                    
    
# Se verifica que la matriz local Ke sea cuadrada y del tamaño de idx
nfil, ncol = Ke.shape             
assert nfil == ncol == len(idx),\
            "Ke debe ser cuadrada y debe tener el mismo número de filas que idx"

       
    
# Se procede con el ensamblaje                 
for i in range(nfil):                             
    for j in range(ncol):                      
    K[idx[i], idx[j]] += Ke[i,j]    
```

- Se verifica que la matriz de rigidez global $\boldsymbol{K}$ sea cuadrada. Si no fuera cuadrada, se arrojaría un error:

In [21]:
nfil, ncol = K.shape                                                    # K.shape es la tupla (4,4), o sea que nfil=4 y ncol=4
assert nfil == ncol, "La matriz de rigidez K debe ser cuadrada"         # En efecto, 4 = 4

- Se verifica que la matriz de rigidez local $\boldsymbol{K}^{(e)}$ sea cuadrada y tenga tantas columnas como el vector `idx`, es decir, $\boldsymbol{K}^{(e)}$ debe ser de tamaño 2x2. En caso contrario, se arrojaría un error, ya que el tamaño de $\boldsymbol{K}^{(e)}$ determina el número de nodos de la barra #$e$, y esto debe ser congruente con el número de nodos por barra en la matriz `LaG`, de donde se extrae el vector `idx`.

In [22]:
nfil, ncol = Ke.shape                                                                    # Ke.shape es la tupla (2,2), o sea que nfil=2 y ncol=2
assert nfil == ncol == len(idx),\
            "Ke debe ser cuadrada y debe tener el mismo número de filas que idx"         # En efecto, 2 = 2 = 2

- El ensamblaje de $\boldsymbol{K}$ consiste de un ciclo `for` dentro de otro ciclo `for`.  
El iterable `i` recorre las filas de $\boldsymbol{K}^{(e)}$, mientras que `j`recorre las columnas.  
A continuación se realizará la iteración $i=2$ y $j=1$ ($i=1$ y $j=0$ en Python) del ciclo `for`, que extraerá la componente $(2, 1)$ de $\boldsymbol{K}^{(2)}$ y la ensamblará en la componente $(3, 2)$ de $\boldsymbol{K}$:

In [23]:
i = 2-1                                              
j = 1-1

# for i in range(nfil):                    Se ejecuta una iteración por cada una de las 2 filas de Ke   
    # for j in range(ncol):                Se ejecuta una iteración por cada una de las 2 columnas de Ke. El ciclo anidado ejecuta 2*2=4 iteraciones.

K[idx[i], idx[j]] += Ke[i, j]            # Se extrae la componente [2,1] de Ke, y se le suma a la matriz  K en la posicón [idx[2], idx[1]] = [3, 2]
                                                
imprimir(r"\boldsymbol{K} = ", K)

<IPython.core.display.Math object>

---

##### 2.4.2.1. Función `ensamblar_vec` para $\boldsymbol{f}$.

En la siguiente línea se utiliza la función ya definida como `ensamblar_vec(v, idx, ve)` con los siguientes parámetros:

- `v`: Vector de fuerzas nodales equivalentes global $\boldsymbol{f}$.
- `ve`: Vector de fuerzas nodales equivalentes local de la barra #$e$, denotado por $\boldsymbol{f}^{(e)}$.

Pero en lugar de ejecutarla, vamos a ejecutar las líneas que hay dentro de la función:
```
def ensamblar_vec(v, idx, ve):

    # Se verifica que v sea un vector columna
    nfil, ncol = v.shape
    assert ncol == 1, "v debe ser un vector columna"           
                                                
    
    
    # Se verifica que el ve sea vector columna y que tenga el mismo número de elementos que idx
    nfil, ncol = ve.shape
    assert ncol == 1 and nfil == len(idx),\
            "ve debe ser vector columna y debe tener el mismo número de elementos que idx"
                                                


    
    # se procede con el ensamblaje              
    for i in range(nfil):                       
        f[idx[i]] += ve[i]                 

```

- Inicialmente, se imprime el vector de fuerzas nodales equivalentes de la barra $e=2$ ($e=1$ en Python), que se obtiene de `fe[:, e]`, pues cada columna de la matriz `fe`representa cada una de las tres barras.

In [24]:
imprimir(r" \boldsymbol{f}^{(2)} = ", fe[:, 2-1] )

<IPython.core.display.Math object>

- Se verifica que el vector de fuerzas nodales equivalentes global $\boldsymbol{f}$ sea un vector columna. En caso de no serlo, se arrojaría un error:

In [25]:
nfil, ncol = f.shape                                      # f.shape es la tupla (4,1), o sea que nfil=4 y ncol=1
assert ncol == 1, "v debe ser un vector columna"          # En efecto, 1 = 1

- Se verifica que el número de filas del vector columna $\boldsymbol{f}^{(2)}$ (o `fe[:, 2-1]`) sea igual que el número de elementos de `idx`, es decir,  $\boldsymbol{f}^{(2)}$ debe tener dos filas. En caso contrario, se arrojaría un error, ya que el tamaño de  $\boldsymbol{f}^{(2)}$ determina el número de nodos de la barra #$e$, y esto debe ser congruente con el número de nodos por barra en la matriz `LaG`, de donde se extrae el vector `idx`.  


In [26]:
nfil, ncol = fe[:, e].shape                                                                     # f.shape es la tupla (2,1), o sea que nfil=2 y ncol=1
assert nfil == len(idx),\
            "ve debe ser vector columna y debe tener el mismo número de elementos que idx"      # En efecto, 2 = 2

- El ensamblaje de $\boldsymbol{f}$ consiste de un ciclo `for`, en el que el iterable `i` recorre los elementos del vector $\boldsymbol{f}^{(e)}$. A continuación se realizará la iteración $i=1$ ($i=0$ en Python), que extraerá el elemento $1$ de $\boldsymbol{f}^{(2)}$ y lo ensamblará en el elemento $2$ de $\boldsymbol{f}$:

In [27]:
i = 1-1

# for i in range(nfil):              Se ejecuta una iteración por cada una de las dos filas de fe[:, e]          

f[idx[i]] += (fe[:, e])[i]           # Se extrae el elemento 1 de fe[:, 2] y se ensambla en el elemento idx[1] = 2 del vector global f

imprimir(r" \boldsymbol{f} = ", f)

<IPython.core.display.Math object>

---

### 2.5. Ensamblaje automático

Una vez comprendido el funcionamiento de las líneas de código para ensamblar $\boldsymbol{K}$ y $\boldsymbol{f}$, se ejecutarán todas a la vez, para luego imprimir la matriz y el vector totalmente ensamblados.

In [28]:
K = sp.zeros(4)                                # Reinicia la matriz K, pues el ejemplo la había alterado

f = sp.zeros(4,1)                              # Reinicia el vector f, pues el ejemplo lo había alterado
f[3-1] = P/2                                   # Se agrega la carga puntual aplicada en el g.d.l. 3
f[4-1] = P                                     # Se agrega la carga puntual aplicada en el g.d.l. 4

for e in range(3):                             # Se ejecuta una iteración para cada barra e = 1, 2 y 3 (0, 1 y 2 en Python)
   idx = LaG[e,:]                              # Vector fila con índices de los nodos globales de la barra e
   Ke  = k[e]*np.array([[1, -1],[-1, 1]])      # Matriz de rigidez local de la barra e
   ensamblar_mat(K, idx, Ke)                   # Se ensambla Ke en la matriz de rigidez global K
   ensamblar_vec(f, idx, fe[:,e])              # Se ensamblan las fuerzas nodales equivalentes de la barra e en el vector global f

- Matriz de rigidez gobal:

In [29]:
imprimir(r" \boldsymbol{K} = ", K)

<IPython.core.display.Math object>

- Vector de fuerzas nodales equivalentes global:

In [30]:
imprimir(r" \boldsymbol{f} = ", f)

<IPython.core.display.Math object>

---

## 3. Solución al sistema de ecuaciones $\boldsymbol{q} = \boldsymbol{K}\boldsymbol{a} - \boldsymbol{f}$.

### 3.1. Grados de libertad conocidos $(\boldsymbol{c})$ y desconocidos $(\boldsymbol{d})$.

Se definen los g.d.l. asociados a los desplazamientos conocidos (c) y desconocidos (d). Recuerde que los g.d.l. conocidos son aquellos en los que los desplazamientos están restringidos (son iguales a 0) y los desconocidos son los g.d.l. restantes: 

In [31]:
c = np.array([1, 2]) - 1                          # Se le resta 1 porque en python el conteo empieza desde 0
c = sp.Matrix(c)                                  # Se convierte de Numpy a SymPy
d = np.setdiff1d(np.arange(4), c)                 # setdiff encuentra la diferencia de conjuntos entre dos arreglos: d = [1,2,3,4] - [1,2] = [3,4]
d = sp.Matrix(d)                                  # Se convierte de Numpy a SymPy

In [32]:
imprimir(r" \boldsymbol{c} = ", c + sp.Matrix([1, 1]))     # Se le suma [1,1] para que empiece el conteo desde 1

<IPython.core.display.Math object>

In [33]:
imprimir(r" \boldsymbol{d} = ", d + sp.Matrix([1, 1]))     # Se le suma [1,1] para que empiece el conteo desde 1

<IPython.core.display.Math object>

### 3.2. Partición de la ecuación $\boldsymbol{q} = \boldsymbol{K}\boldsymbol{a} - \boldsymbol{f}$.

Luego de particionar la ecuación $\boldsymbol{q} = \boldsymbol{K}\boldsymbol{a} - \boldsymbol{f}$ de la siguiente manera
$$\underbrace{\begin{bmatrix}  \boldsymbol{q}_{d} \\ \boldsymbol{q}_{c}  \end{bmatrix}}_{\boldsymbol{q}}  = 
  \underbrace{\begin{bmatrix}  \boldsymbol{K}_{cc} & \boldsymbol{K}_{cd}  \\  \boldsymbol{K}_{dc} & \boldsymbol{K}_{dd} \end{bmatrix}}_{\boldsymbol{K}}
  \underbrace{\begin{bmatrix}  \boldsymbol{a}_{c} \\ \boldsymbol{a}_{d}  \end{bmatrix}}_{\boldsymbol{a}}  -
  \underbrace{\begin{bmatrix}  \boldsymbol{f}_{d} \\ \boldsymbol{f}_{c}  \end{bmatrix}}_{\boldsymbol{f}} ,
$$

se definen los respectivos subvectores y submatrices de los vectores $\boldsymbol{a}$, $\boldsymbol{f}$ y la matriz $\boldsymbol{K}$ (exceptuando $\boldsymbol{a}_{d}$ y $\boldsymbol{q}_{d}$ que son incógnitas, y $\boldsymbol{q}_{c} = \boldsymbol{0}$ que no es necesario para solucionar el sistema de ecuaciones):

In [34]:
Kcc = K.extract(c,c)        # Submatriz con los elementos de las filas {1,2} y las columnas {1,2} de la matriz K
Kcd = K.extract(c,d)        # Submatriz con los elementos de las filas {1,2} y las columnas {3,4} de la matriz K
Kdc = K.extract(d,c)        # Submatriz con los elementos de las filas {3,4} y las columnas {1,2} de la matriz K
Kdd = K.extract(d,d)        # Submatriz con los elementos de las filas {3,4} y las columnas {3,4} de la matriz K
ac  = sp.Matrix([0, 0])     # Normalmente, el vector de desplazamientos conocidos es nulo. En este caso ac tiene dos elementos ya que c tiene dos g.d.l.
#ad = a.extract(d,[0])      # Subvector con los elementos 3 y 4 del vector a
fc  = f.extract(d,[0])      # Subvector con los elementos 3 y 4 del vector f (recuerde que fc está asociado a los g.d.l. desconocidos d)
fd  = f.extract(c,[0])      # Subvector con los elementos 1 y 2 del vector f (recuerde que fd está asociado a los g.d.l. conocidos c)

In [35]:
imprimir(r"\boldsymbol{K}_{cc} = ", Kcc)

<IPython.core.display.Math object>

In [36]:
imprimir(r"\boldsymbol{K}_{cd} = ", Kcd)

<IPython.core.display.Math object>

In [37]:
imprimir(r"\boldsymbol{K}_{dc} = ", Kdc)

<IPython.core.display.Math object>

In [38]:
imprimir(r"\boldsymbol{K}_{dd} = ", Kdd)

<IPython.core.display.Math object>

In [39]:
imprimir(r"\boldsymbol{a}_{c} = ", ac)

<IPython.core.display.Math object>

In [40]:
imprimir(r"\boldsymbol{f}_{c} = ", fc)

<IPython.core.display.Math object>

In [41]:
imprimir(r"\boldsymbol{f}_{d} = ", fd)

<IPython.core.display.Math object>

### 3.3. Cálculo de fuerzas de equilibrio y desplazamientos desconocidos.

Se calculan los vectores $\boldsymbol{a}_{d}$ y $\boldsymbol{q}_{d}$ por medio de las siguientes ecuaciones:
        $$\boldsymbol{a}_{d} = \boldsymbol{K}_{dd}^{-1} ( \boldsymbol{f}_{c} - \boldsymbol{K}_{dc} \boldsymbol{a}_{c} )$$
        $$\boldsymbol{q}_{d} = \boldsymbol{K}_{cc} \boldsymbol{a}_{c} + \boldsymbol{K}_{cd} \boldsymbol{a}_{d} - \boldsymbol{f}_{d}$$

In [42]:
ad = Kdd.solve(fc - Kdc*ac)
qd = Kcc*ac + Kcd*ad - fd

In [43]:
imprimir(r"\boldsymbol{a}_d = ", ad)

<IPython.core.display.Math object>

In [44]:
imprimir(r"\boldsymbol{q}_d = ", qd)

<IPython.core.display.Math object>

---

## 4. Ensamblaje de los vectores $\boldsymbol{a}$ y $\boldsymbol{q}$.

Se ensambla el vector de desplazamientos nodales global $(\boldsymbol{a})$ y el vector de fuerzas nodales de equilibrio $(\boldsymbol{q})$, recordando que
        $$ \boldsymbol{a} = \begin{bmatrix}  \boldsymbol{a}_{c} \\ \boldsymbol{a}_{d}  \end{bmatrix} \hspace{4cm}
           \boldsymbol{q} = \begin{bmatrix}  \boldsymbol{q}_{d} \\ \boldsymbol{q}_{c}  \end{bmatrix}                   $$


In [45]:
a = sp.zeros(4,1); q = sp.zeros(4,1)     # Se separa memoria para a y q, ambos de tamaño 4x1 porque la estructura tiene cuatro g.d.l.

# Se ensamblan los elementos en asociados a g.d.l. conocidos:
for i in range(len(c)):            # Se ejecutan dos iteraciones, una para cada g.d.l. conocido. Por ejempo para i = 1:
    a[c[i]] = ac[i]                # Se extrae el elemento 1 de ac y se ensambla en el elemento c[1] = 1 del vector a.
    q[c[i]] = qd[i]                # Se extrae el elemento 1 de qd y se ensambla en el elemento c[1] = 1 del vector q. 
                                   # Recuerde que qd está asociado a los g.d.l. conocidos

In [46]:
# Se ensamblan los elementos asociados a g.d.l. desconocidos:
for i in range(len(d)):             # Se ejecutan dos iteraciones, una para cada g.d.l. conocido. Por ejempo para i = 2:
    a[d[i]] = ad[i]                 # Se extrae el elemento 2 de ad y se ensambla en el elemento d[2] = 4 del vector a.
  # q[d[i]] = qc[i] = 0             # Se extrae el elemento 2 de qc y se ensambla en el elemento d[2] = 4 del vector q. Es nnecesario pues qc = 0
                                    # Recuerde que qc está asociado a los g.d.l. desconocidos

Se simplifican las expresiones dentro de $\boldsymbol{a}$ y $\boldsymbol{q}$:

In [47]:
a   = sp.simplify(a)
q   = sp.simplify(q)

Finalmente se imprimen estos dos vectores:
  - Vector de desplazamientos nodales global $\boldsymbol{a}$.
  - Vector de fuerzas nodales de equilibrio global $\boldsymbol{q}$.

In [48]:
imprimir(r" \boldsymbol{a} = ", a)

<IPython.core.display.Math object>

In [49]:
imprimir(r" \boldsymbol{q} = ", q)

<IPython.core.display.Math object>

---

## 5. Cálculo de fuerzas axiales.

Se calculan las fuerzas axiales en cada barra mediante la fórmula
                                  $$ f^{(e)}_{ax} = k^{e} [ u^{(e)}_{2} - u^{(e)}_{2} ] .$$

In [50]:
fax = sp.zeros(3,1)             # Se separa memoria con un vector de ceros de tamaño 3x1 (una componente para cada barra)

for e in range(3):              # Se ejecuta una iteración para cada barra e = 1, 2 y 3 (0, 1 y 2 en Python)                 
   fax[e] = k[e]*(a[LaG[e,NL2]] - a[LaG[e,NL1]])

donde:
- `fax[e]`        : El resultado de la fuerza axial de la barra #$e$, se guarda en la posicón $e$ del vector `fax` (esto es $f^{(e)}_{ax}$).
   
- `k[e]`          : Elemento #$e$ de la lista `k`, que representa la rigidez axial de la barra #$e$ (esto es $k^{(e)}$).
 
- `LaG[e,NL2]`    : Nodo global que corresponde al nodo local $2$ de la barra #$e$.

- `a[LaG[e,NL2]]` : Desplazamiento del nodo local $2$ de la barra #$e$ (esto es $u^{(e)}_{2}$).
 
- `LaG[e,NL1]`    : Nodo global que corresponde al nodo local $1$ de la barra #$e$.
   
- `a[LaG[e,NL1]]` : Desplazamiento del nodo local $1$ de la barra #$e$. (esto es $u^{(e)}_{1}$).

Se simplifican las expresiones dentro del vector obtenido $\boldsymbol{f}_{ax}$:

In [51]:
fax = sp.simplify(fax)

Y finalmente se imprime el vector de fuerzas axiales $\boldsymbol{f}_{ax}$, en el que la componente $e$ corresponde a $f^{(e)}_{ax}$:

In [52]:
imprimir(r" \boldsymbol{f}_{ax} = ", fax)

<IPython.core.display.Math object>

In [53]:
imprimir(r" f_{ax}^{(1)} = ", fax[1-1])

<IPython.core.display.Math object>

In [54]:
imprimir(r" f_{ax}^{(2)} = ", fax[2-1])

<IPython.core.display.Math object>

In [55]:
imprimir(r" f_{ax}^{(3)} = ", fax[3-1])

<IPython.core.display.Math object>